In [1]:
import os
import sys
import glob
from os import write

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities
from iDrink.iDrinkUtilities import get_title_measure_name, get_unit, get_cad, get_setting_axis_name

In [2]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_stat_cat = os.path.join(root_stat, '02_categorical')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots_cont = os.path.join(dir_stat_cont, '02_plots')
dir_plots_cat = os.path.join(dir_stat_cat, '02_plots')

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')
df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']
idx_s_singlecam_full = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']
idx_s_singlecam = ['S017', 'S018']
idx_s_multicam = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']
idx_s_multicam_reduced = ['S001', 'S002']
idx_s_reduced = idx_s_multicam_reduced + idx_s_singlecam
idx_s_full = idx_s_multicam + idx_s_singlecam

In [26]:
csv_offset = os.path.join(root_data, 'preprocessed_data', 'offset.csv')
df_offset = pd.read_csv(csv_offset, sep=';')

In [27]:
reduced_analysis = True # If True, only S001 and S002 from Multicam will be used.
if reduced_analysis:
    file_app = '_reduced'
    df_offset = df_offset[(df_offset['id_s'].isin(idx_s_reduced)) & (~df_offset['id_p'].isin(ignore_id_p))]
else:
    file_app = ''

In [33]:
df_latex = df_offset.groupby(['id_s'], as_index=False).mean(numeric_only=True)


In [34]:
df_latex.insert(0, 'Setting', df_latex['id_s'].apply(lambda x: get_setting_axis_name(x)))

In [35]:
df_latex.drop(columns=['id_s'], inplace=True)

In [42]:
df_latex.round(2).set_index('Setting').T

Setting,"SimCC, Cams: 1,2,3,4,5","Metrabs, Cams: 1,2,3,4,5","Single, Cam: 1, filt","Single, Cam: 1, unfilt"
hand_vel,658.50,548.67,453.63,761.35
elbow_vel,0.90,1.08,0.94,1.02
trunk_disp,65.55,55.66,62.56,62.59
trunk_ang,323.79,327.84,373.70,373.71
elbow_flex_pos,182.81,170.35,172.88,172.88
shoulder_flex_pos,102.78,106.37,104.81,104.81
shoulder_abduction_pos,46.22,29.60,30.63,30.63


In [23]:
for old_col in df_latex.columns:
    new_col = get_title_measure_name(old_col, add_unit = True)

    if type(new_col) == str:
        df_latex.rename(columns={old_col: new_col}, inplace=True)

In [24]:
df_latex.round(2)

,Setting,id_p,Hand Velocity [mm/s],Elbow Velocity [deg/s],Trunk Displacement [mm],Trunk Angle [deg],Elbow Flexion [deg],Shoulder Flexion [deg],Shoulder Abduction [deg]
0,"SimCC, Cams: 1,2,3,4,5",P07,364.26,0.60,53.96,359.27,172.10,89.91,32.98
1,"SimCC, Cams: 1,2,3,4,5",P08,338.59,0.72,26.67,353.83,186.29,82.63,36.79
2,"SimCC, Cams: 1,2,3,4,5",P10,373.74,0.92,79.63,255.06,179.26,118.18,55.36
3,"SimCC, Cams: 1,2,3,4,5",P12,571.78,1.27,29.82,239.48,187.38,112.98,52.63
4,"SimCC, Cams: 1,2,3,4,5",P15,2216.35,0.71,108.29,370.09,161.76,109.33,44.73
5,"SimCC, Cams: 1,2,3,4,5",P241,362.21,0.59,59.53,363.83,206.58,106.75,55.97
6,"SimCC, Cams: 1,2,3,4,5",P242,373.82,0.62,123.32,360.42,196.29,101.26,52.34
7,"SimCC, Cams: 1,2,3,4,5",P251,332.73,0.88,44.49,364.07,188.43,103.29,42.83
8,"SimCC, Cams: 1,2,3,4,5",P252,362.89,0.91,39.24,358.73,188.92,100.64,40.61
9,"Metrabs, Cams: 1,2,3,4,5",P07,374.09,0.59,56.30,359.97,157.85,95.71,17.26


In [25]:
print(df_latex.round(2).style.to_latex(sparse_index=False,
                                       label = 'tab:offset_reduced',
                                       caption = r'Offset values for the reduced set of settings. The full table can be found in section\ref{app:additional_ts_tables}.'))

\begin{table}
\caption{Offset values for the reduced set of settings. The full table can be found in section\ref{app:additional_ts_tables}.}
\label{tab:offset_reduced}
\begin{tabular}{lllrrrrrrr}
 & Setting & id_p & Hand Velocity [mm/s] & Elbow Velocity [deg/s] & Trunk Displacement [mm] & Trunk Angle [deg] & Elbow Flexion [deg] & Shoulder Flexion [deg] & Shoulder Abduction [deg] \\
0 & SimCC, Cams: 1,2,3,4,5 & P07 & 364.260000 & 0.600000 & 53.960000 & 359.270000 & 172.100000 & 89.910000 & 32.980000 \\
1 & SimCC, Cams: 1,2,3,4,5 & P08 & 338.590000 & 0.720000 & 26.670000 & 353.830000 & 186.290000 & 82.630000 & 36.790000 \\
2 & SimCC, Cams: 1,2,3,4,5 & P10 & 373.740000 & 0.920000 & 79.630000 & 255.060000 & 179.260000 & 118.180000 & 55.360000 \\
3 & SimCC, Cams: 1,2,3,4,5 & P12 & 571.780000 & 1.270000 & 29.820000 & 239.480000 & 187.380000 & 112.980000 & 52.630000 \\
4 & SimCC, Cams: 1,2,3,4,5 & P15 & 2216.350000 & 0.710000 & 108.290000 & 370.090000 & 161.760000 & 109.330000 & 44.730000 \\


In [33]:
print(df_latex.to_markdown())

|    | Setting []               | id_p []   |   Hand Velocity [mm/s] |   Elbow Velocity [deg/s] |   Trunk Displacement [mm] |   Trunk Angle [deg] |   Elbow Flexion [deg] |   Shoulder Flexion [deg] |   Shoulder Abduction [deg] |
|---:|:-------------------------|:----------|-----------------------:|-------------------------:|--------------------------:|--------------------:|----------------------:|-------------------------:|---------------------------:|
|  0 | SimCC, Cams: 1,2,3,4,5   | P07       |                365.337 |                 0.770144 |                   54.7249 |             358.971 |               172.938 |                  90.4051 |                    33.6553 |
|  1 | SimCC, Cams: 1,2,3,4,5   | P08       |                338.558 |                 0.72756  |                   28.0976 |             351.088 |               186.834 |                  82.634  |                    41.1836 |
|  2 | SimCC, Cams: 1,2,3,4,5   | P10       |                369.075 |                 0